#### Set conditions for the plots and import the relevant packages

In [ ]:
%matplotlib inline

# plot configuration
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.style.use("ggplot")

matplotlib.rcParams['figure.figsize']  = (7.0,5.0)

In [ ]:
import numpy as np
from scipy import special, stats
import sobol_seq
import pandas as pd
from statsmodels.tsa.arima_model import ARIMA
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from openpyxl import load_workbook
import types
import pkg_resources

### Check the packages requirements for reproducibility

In [ ]:
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]
            
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

### Define base functions and the sample quasi-random distribution 

In [ ]:
def decreasePaceSilver2050(silverCell):
    return - 1/(2050-2017) * np.log(silverCell/ITRPV2018silverCell2017)

def mu(x):
    return np.mean(x)

def sigma(x):
    return np.std(x)

In [ ]:
sampleTot = sobol_seq.i4_sobol_generate(13,1000*2**13).T

### Current use of silver paste (mg/cell) - year 2017 - source ITRPV 2018

In [ ]:
ITRPV2018silverCell2017 = 100
ITRPVsilverCell2028 = 50 
ITRPV2018decreasePace= - 1/(2028-2017) * np.log(ITRPVsilverCell2028/ITRPV2018silverCell2017)

### Silver cell decrease pace - distribution generated on the basis of the experts consultation

In [ ]:
silverCell2050DecreasePace = np.array([ITRPV2018decreasePace, ITRPV2018decreasePace, \
decreasePaceSilver2050(ITRPV2018silverCell2017*0.01), decreasePaceSilver2050(ITRPV2018silverCell2017*0.01),\
decreasePaceSilver2050(10), decreasePaceSilver2050(15),decreasePaceSilver2050((20+10)/2), 0.05])

### PV Cell Efficiency growth up to the year 2050 (W/cell)

In [ ]:
PVCellEfficiency2017 = 4.27

PVCellEfficiency2050 = 6.1 

PVCellEfficiencyIncreasePace = 1/(2050-2017) * np.log(PVCellEfficiency2050/PVCellEfficiency2017)

### Current expansion of PV power capacity (GW) - year 2017

In [ ]:
PVPC2017 = 90
CrystallineSiliconShare = 0.9
IEA2050 = 4670
Greenpeace2050 = 9295
Fraunhofer2050 = 30700
CPVPC2017 = 402

IEA2050_CrystallineSiliconShare = IEA2050 * CrystallineSiliconShare
Greenpeace2050_CrystallineSiliconShare = Greenpeace2050 * CrystallineSiliconShare
Fraunhofer2050_CrystallineSiliconShare = Fraunhofer2050 * CrystallineSiliconShare
PVPC2017_CrystallineSiliconShare = PVPC2017 * CrystallineSiliconShare

PVPC2050IncreasePace = np.array([0.025, 0.112, 0.061])

### Non-industrial + Industrial Non-PV

In [ ]:
wb = load_workbook(filename='Datum.xlsx', read_only=True)

ws = wb['World_silver_survey_2018']

# Read the cell values into a list of lists
data_rows = []
for row in ws['B26':'M26']:
    data_cols = []
    for cell in row:
        data_cols.append(cell.value)
    data_rows.append(data_cols)

otherAg = pd.DataFrame(data_rows).T
otherAg.index=[y for y in range(2006,2018)]
# A negative correlation between the two variables has been confirmed by checking the sign of the anti-diagonal matrix

### Yearly extraction capacity

In [ ]:
Extraction = pd.read_excel('Datum.xlsx',sheet_name='USGS',index_col=0)
Extraction.columns = ['Global Production']

### Generate the parameters distribution 

In [ ]:
silverCellDistribution2 = pd.DataFrame(sigma(silverCell2050DecreasePace)*2**0.5*special.erfinv(2*(sampleTot[0][:2**13]*
0.5*(special.erf((silverCell2050DecreasePace.max()-mu(silverCell2050DecreasePace))/(sigma(silverCell2050DecreasePace)*2**0.5))-
special.erf(-mu(silverCell2050DecreasePace)/(sigma(silverCell2050DecreasePace)*2**0.5)))+0.5*
special.erf(-mu(silverCell2050DecreasePace)/(sigma(silverCell2050DecreasePace)*2**0.5))))+mu(silverCell2050DecreasePace))

silverCellDistribution2B = pd.DataFrame(sigma(silverCell2050DecreasePace)*2**0.5*special.erfinv(2*(sampleTot[6][:2**13]*
0.5*(special.erf((silverCell2050DecreasePace.max()-mu(silverCell2050DecreasePace))/(sigma(silverCell2050DecreasePace)*2**0.5))-
special.erf(-mu(silverCell2050DecreasePace)/(sigma(silverCell2050DecreasePace)*2**0.5)))+0.5*
special.erf(-mu(silverCell2050DecreasePace)/(sigma(silverCell2050DecreasePace)*2**0.5))))+mu(silverCell2050DecreasePace))

PVPC2050IncreasePaceDist = pd.DataFrame(pd.DataFrame(sigma(PVPC2050IncreasePace)*2**0.5*special.erfinv(2*(sampleTot[1][:2**13]*
0.5*(special.erf((PVPC2050IncreasePace.max()-mu(PVPC2050IncreasePace))/(sigma(PVPC2050IncreasePace)*2**0.5))-
special.erf(-mu(PVPC2050IncreasePace)/(sigma(PVPC2050IncreasePace)*2**0.5)))+0.5*
special.erf(-mu(PVPC2050IncreasePace)/(sigma(PVPC2050IncreasePace)*2**0.5))))+mu(PVPC2050IncreasePace)))

PVPC2050IncreasePaceDistB = pd.DataFrame(pd.DataFrame(sigma(PVPC2050IncreasePace)*2**0.5*special.erfinv(2*(sampleTot[7][:2**13]*
0.5*(special.erf((PVPC2050IncreasePace.max()-mu(PVPC2050IncreasePace))/(sigma(PVPC2050IncreasePace)*2**0.5))-
special.erf(-mu(PVPC2050IncreasePace)/(sigma(PVPC2050IncreasePace)*2**0.5)))+0.5*
special.erf(-mu(PVPC2050IncreasePace)/(sigma(PVPC2050IncreasePace)*2**0.5))))+mu(PVPC2050IncreasePace)))

Noise = pd.DataFrame(sigma(sampleTot[2][:2**13])*2**0.5*special.erfinv(2*sampleTot[2][:2**13]-1))
NoiseB = pd.DataFrame(sigma(sampleTot[8][:2**13])*2**0.5*special.erfinv(2*sampleTot[8][:2**13]-1))

# Resources and reserves
distributionSilverReserveResource = pd.DataFrame(sampleTot[4][:2**13]*(4000-530)+530)
distributionSilverReserveResourceB = pd.DataFrame(sampleTot[10][:2**13]*(4000-530)+530)

### Fit the non-PV Silver use and Silver extraction models

In [ ]:
model = ARIMA((otherAg.reset_index(drop=True)+Noise.iloc[:len(otherAg)])[0], order=(0, 1, 1))
model_fit = model.fit(disp=False)
# make prediction
yhat = model_fit.predict(len(otherAg), 44, typ='levels')
yhat.index = yhat.index + 2006
forecast, stderr, conf = model_fit.forecast(33)

modelB = ARIMA((otherAg.reset_index(drop=True)+NoiseB.iloc[:len(otherAg)])[0], order=(0, 1, 1))
model_fitB = modelB.fit(disp=False)
# make prediction
yhatB = model_fitB.predict(len(otherAg), 44, typ='levels')
yhatB.index = yhat.index
forecastB, stderrB, confB = model_fitB.forecast(33)

model2 = ARIMA(((Extraction['Global Production'].reset_index(drop=True)+Noise.iloc[:len(Extraction)].T).T)[0],
               order=(0, 2, 1))
model_fit2 = model2.fit(disp=False)
# make prediction
yhat2 = model_fit2.predict(len(Extraction), 105, typ='levels')
yhat2.index = yhat2.index+1946
forecast2, stderr2, conf2 = model_fit2.forecast(33)

model2B = ARIMA(((Extraction['Global Production'].reset_index(drop=True)+Noise.iloc[:len(Extraction)].T).T)[0], 
                 order=(0, 2, 1))
model_fit2B = model2B.fit(disp=False)
# make prediction
yhat2B = model_fit2B.predict(len(Extraction), 105, typ='levels')
yhat2B.index = yhat2B.index+1946
forecast2B, stderr2B, conf2B = model_fit2B.forecast(33)

### Generate the distributions for non-PV Silver use and Silver extraction

In [ ]:
otherAgDist = pd.DataFrame([((forecast[i]-conf[i][0])/1.96)*2**0.5*special.erfinv(sampleTot[3][:2**13]*
(1-special.erf(-forecast[i]/(((forecast[i]-conf[i][0])/1.96)*2**0.5)))+special.erf(-forecast[i]/
(((forecast[i]-conf[i][0])/1.96)*2**0.5)))+forecast[i] for i in range(len(forecast))],index=[i for i in range(2018,2051)])

otherAgDistB = pd.DataFrame([((forecastB[i]-confB[i][0])/1.96)*2**0.5*special.erfinv(sampleTot[9][:2**13]*
(1-special.erf(-forecastB[i]/(((forecastB[i]-confB[i][0])/1.96)*2**0.5)))+special.erf(-forecastB[i]/
(((forecastB[i]-confB[i][0])/1.96)*2**0.5)))+forecastB[i] for i in range(len(forecastB))],index=[i for i in range(2018,2051)])

ExtractionDist = pd.DataFrame([((forecast2[i]-conf2[i][0])/1.96)*2**0.5*special.erfinv(sampleTot[5][:2**13]*
(1-special.erf(-forecast2[i]/(((forecast2[i]-conf2[i][0])/1.96)*2**0.5)))+special.erf(-forecast2[i]/
(((forecast2[i]-conf2[i][0])/1.96)*2**0.5)))+forecast2[i] for i in range(len(forecast2))],index = otherAgDist.index)

ExtractionDistB = pd.DataFrame([((forecast2B[i]-conf2B[i][0])/1.96)*2**0.5*special.erfinv(sampleTot[11][:2**13]*
(1-special.erf(-forecast2B[i]/(((forecast2B[i]-conf2B[i][0])/1.96)*2**0.5)))+special.erf(-forecast2B[i]/
(((forecast2B[i]-conf2B[i][0])/1.96)*2**0.5)))+forecast2B[i] for i in range(len(forecast2B))],index = otherAgDistB.index)

#### Assess PV silver demand, Yearly Silver demand (and its derivative), Cumulative Silver demand along with viability and feasibility

In [ ]:
# Generation of the yearly silver output (1,000 metric tonnes)
PVDemand = pd.concat([(ITRPV2018silverCell2017 * np.exp(-silverCellDistribution2*i1)/(PVCellEfficiency2017 * 
np.exp(PVCellEfficiencyIncreasePace*i1)))* PVPC2017 * CrystallineSiliconShare * np.exp(PVPC2050IncreasePaceDist*i1)/
1000 for i1 in range(33)],axis=1)

PVDemand.columns = [i2 for i2 in range(2018,2051)]

PVDemandAB1 = pd.concat([(ITRPV2018silverCell2017 * np.exp(-silverCellDistribution2B*i1)/(PVCellEfficiency2017 * 
np.exp(PVCellEfficiencyIncreasePace*i1)))* PVPC2017 * CrystallineSiliconShare * np.exp(PVPC2050IncreasePaceDist*i1)/
1000 for i1 in range(33)],axis=1)

PVDemandAB1.columns = [i2 for i2 in range(2018,2051)]

PVDemandAB2 = pd.concat([(ITRPV2018silverCell2017 * np.exp(-silverCellDistribution2*i1)/(PVCellEfficiency2017 * 
np.exp(PVCellEfficiencyIncreasePace*i1)))* PVPC2017 * CrystallineSiliconShare * np.exp(PVPC2050IncreasePaceDistB*i1)/
1000 for i1 in range(33)],axis=1)

PVDemandAB2.columns = [i2 for i2 in range(2018,2051)]

PVDemandB = pd.concat([(ITRPV2018silverCell2017 * np.exp(-silverCellDistribution2B*i1)/(PVCellEfficiency2017 * 
np.exp(PVCellEfficiencyIncreasePace*i1)))* PVPC2017 * CrystallineSiliconShare * np.exp(PVPC2050IncreasePaceDistB*i1)/
1000 for i1 in range(33)],axis=1)

PVDemandB.columns = [i2 for i2 in range(2018,2051)]

# Generation of the cumulative silver output (1,000 metric tonnes)
yearlySilver = PVDemand+otherAgDist.T
yearlySilverAB1 = PVDemandAB1+otherAgDist.T
yearlySilverAB2 = PVDemandAB2+otherAgDist.T
yearlySilverAB3 = PVDemand+otherAgDistB.T
yearlySilverB = PVDemandB+otherAgDistB.T

CumulativeSilver = yearlySilver.cumsum(axis=1)
CumulativeSilverAB1 = yearlySilverAB1.cumsum(axis=1)
CumulativeSilverAB2 = yearlySilverAB2.cumsum(axis=1)
CumulativeSilverAB3 = yearlySilverAB3.cumsum(axis=1)
CumulativeSilverB = yearlySilverB.cumsum(axis=1)

# Derivative Yearly Silver Demand
derivativeYearlySilver = yearlySilver.diff(axis=1)

# Yearly Delta (viability, technological constraint)

silverDelta = ExtractionDist.T - 0.8*yearlySilver

# Cumulative Delta (feasibility, biophysical constraint)

Difference = distributionSilverReserveResource.values-0.8*CumulativeSilver
DifferenceAB1 = distributionSilverReserveResource.values-0.8*CumulativeSilverAB1
DifferenceAB2 = distributionSilverReserveResource.values-0.8*CumulativeSilverAB2
DifferenceAB3 = distributionSilverReserveResource.values-0.8*CumulativeSilverAB3
DifferenceAB4 = distributionSilverReserveResourceB.values-0.8*CumulativeSilver
DifferenceB = distributionSilverReserveResourceB.values-0.8*CumulativeSilverB

### Bootstrap Sensitivity indices 

In [ ]:
sM_S = [yearlySilver,yearlySilverAB1,yearlySilverAB2,yearlySilverAB3,yearlySilverB]

S_silverCell = []
S_PVPC = []
S_otherAg = []
T_silverCell = []
T_PVPC = []
T_otherAg = []

for r in range(1000):
    seed = (sampleTot[-1,r*len(yearlySilver):(r+1)*len(yearlySilver)]*len(yearlySilver)).astype(int)
    
    sM_BS = [smS.merge(pd.DataFrame(index=seed), left_index=True, right_index=True, how='right') for smS in sM_S]
    
    Var = pd.concat([sM_BS[0],sM_BS[-1]]).var(ddof=0)
    
    S_silverCell.append(((Var-0.5*((sM_BS[-1]-sM_BS[1])**2).mean())/Var).values)
    S_PVPC.append(((Var-0.5*((sM_BS[-1]-sM_BS[2])**2).mean())/Var).values)
    S_otherAg.append(((Var-0.5*((sM_BS[-1]-sM_BS[3])**2).mean())/Var).values)
    T_silverCell.append(((0.5*((sM_BS[0]-sM_BS[1])**2).mean())/Var).values)
    T_PVPC.append(((0.5*((sM_BS[0]-sM_BS[2])**2).mean())/Var).values)
    T_otherAg.append(((0.5*((sM_BS[0]-sM_BS[3])**2).mean())/Var).values)

cln = [yr for yr in range(2018,2051)]
S_silverCell_df = pd.DataFrame(S_silverCell,columns=cln)
S_PVPC_df = pd.DataFrame(S_PVPC,columns=cln)
S_otherAg_df = pd.DataFrame(S_otherAg,columns=cln)
T_silverCell_df = pd.DataFrame(T_silverCell,columns=cln)
T_PVPC_df = pd.DataFrame(T_PVPC,columns=cln)
T_otherAg_df = pd.DataFrame(T_otherAg,columns=cln)

### Get the 2050 figures for the Smirnov-Kolmorov test

In [ ]:
df20503d = pd.concat([silverCellDistribution2,PVPC2050IncreasePaceDist,pd.DataFrame([df2050[2050] for df2050 in 
            [otherAgDist.T,yearlySilver,CumulativeSilver,silverDelta,Difference]]).T],axis=1)

df20503d.columns = ['silverCell','PVPC','nonPVsilver','yearlySilver','CumulativeSilver','yearly Delta','Difference']

In [ ]:
BVsilverCell=df20503d['silverCell'][df20503d['yearly Delta']>0].sort_values()
BVsilverCellCum = BVsilverCell.cumsum()/BVsilverCell.sum()
NBVsilverCell=df20503d['silverCell'][df20503d['yearly Delta']<0].sort_values()
NBVsilverCellCum = NBVsilverCell.cumsum()/NBVsilverCell.sum()

ax1 = plt.plot(BVsilverCell,BVsilverCellCum,label='silverCell_viable',c='b')
plt.plot(NBVsilverCell,NBVsilverCellCum,label='silverCell_not_viable',c='r')
plt.legend()

In [ ]:
BFsilverCell=df20503d['silverCell'][df20503d['Difference']>0].sort_values()
BFsilverCellCum = BFsilverCell.cumsum()/BFsilverCell.sum()
NBFsilverCell=df20503d['silverCell'][df20503d['Difference']<0].sort_values()
NBFsilverCellCum = NBFsilverCell.cumsum()/NBFsilverCell.sum()

ax1 = plt.plot(BFsilverCell,BFsilverCellCum,label='silverCell_feasible',c='b')
plt.plot(NBFsilverCell,NBFsilverCellCum,label='silverCell_not_feasible',c='r')
plt.legend()

In [ ]:
BVPVPC=df20503d['PVPC'][df20503d['yearly Delta']>0].sort_values()
BVPVPCCum = BVPVPC.cumsum()/BVPVPC.sum()
NBVPVPC=df20503d['PVPC'][df20503d['yearly Delta']<0].sort_values()
NBVPVPCCum = NBVPVPC.cumsum()/NBVPVPC.sum()

ax1 = plt.plot(BVPVPC,BVPVPCCum,label='PVPC_viable',c='b')
plt.plot(NBVPVPC,NBVPVPCCum,label='PVPC_not_viable',c='r')
plt.legend()

In [ ]:
BFPVPC=df20503d['PVPC'][df20503d['Difference']>0].sort_values()
BFPVPCCum = BFPVPC.cumsum()/BFPVPC.sum()
NBFPVPC=df20503d['PVPC'][df20503d['Difference']<0].sort_values()
NBFPVPCCum = NBFPVPC.cumsum()/NBFPVPC.sum()

ax1 = plt.plot(BFPVPC,BFPVPCCum,label='PVPC_feasible',c='b')
plt.plot(NBFPVPC,NBFPVPCCum,label='PVPC_not_feasible',c='r')
plt.legend()

In [ ]:
#Smirnov-Kolmorov Test

BVnonPVsilver=df20503d['nonPVsilver'][df20503d['yearly Delta']>0].sort_values()
BVnonPVsilverCum = BVnonPVsilver.cumsum()/BVnonPVsilver.sum()
NBVnonPVsilver=df20503d['nonPVsilver'][df20503d['yearly Delta']<0].sort_values()
NBVnonPVsilverCum = NBVnonPVsilver.cumsum()/NBVnonPVsilver.sum()

ax1 = plt.plot(BVnonPVsilver,BVnonPVsilverCum,label='nonPVsilver_viable',c='b')
plt.plot(NBVnonPVsilver,NBVnonPVsilverCum,label='nonPVsilver_not_viable',c='r')
plt.legend()

In [ ]:
#Smirnov-Kolmorov Test

BFnonPVsilver=df20503d['nonPVsilver'][df20503d['Difference']>0].sort_values()
BFnonPVsilverCum = BFnonPVsilver.cumsum()/BFnonPVsilver.sum()
NBFnonPVsilver=df20503d['nonPVsilver'][df20503d['Difference']<0].sort_values()
NBFnonPVsilverCum = NBFnonPVsilver.cumsum()/NBFnonPVsilver.sum()

ax1 = plt.plot(BFnonPVsilver,BFnonPVsilverCum,label='nonPVsilver_feasible',c='b')
plt.plot(NBFnonPVsilver,NBFnonPVsilverCum,label='nonPVsilver_not_feasible',c='r')
plt.legend()

### Analysing the distributions over time

In [ ]:
for col in yearlySilver:
    plt.plot([col for row in range(len(yearlySilver))], yearlySilver[col], c = 'r')
    plt.xlabel('Year')
    plt.ylabel('Yearly silver demand (kton)')
    plt.ylim(25,85)
plt.show()

In [ ]:
sns.distplot(yearlySilver[2050],vertical=True,color='r')
plt.xlabel('Relative frequency')
plt.ylabel('2050 Yearly silver demand (kton)')
plt.ylim(25,85)

In [ ]:
for col in PVDemand:
    plt.plot([col for row in range(len(PVDemand))], PVDemand[col], c = 'y')
    plt.xlabel('Year')
    plt.ylabel('Yearly PV silver demand (kton)')
    plt.ylim(-5,45)
plt.show()

In [ ]:
sns.distplot(PVDemand[2050],vertical=True,color='y')
plt.xlabel('Relative frequency')
plt.ylabel('2050 Yearly PV silver demand (kton)')
plt.ylim(-5,45)

In [ ]:
for col in derivativeYearlySilver:
    plt.plot([col for row in range(len(derivativeYearlySilver))], derivativeYearlySilver[col], c = 'k')
    plt.xlabel('Year')
    plt.ylabel('Yearly derivative silver demand (kton)')
plt.show()

In [ ]:
for col in silverDelta:
    plt.plot([col for row in range(len(silverDelta))], silverDelta[col], c = 'c')
    plt.xlabel('Year')
    plt.ylabel('Yearly difference (kton)')
    plt.ylim(-30,30)
plt.show()

In [ ]:
sns.distplot(silverDelta[2050],vertical=True,color='c')
plt.xlabel('Relative frequency')
plt.ylabel('2050 - Yearly difference (kton)')
plt.ylim(-30,30)

In [ ]:
for col in Difference:
    plt.plot([col for row in range(len(Difference))], Difference[col], c = 'w')
    plt.scatter(col, Difference[col].mean(), c = 'm')
    plt.xlabel('Year')
    plt.ylabel('Cumulative difference (kton)')
    plt.ylim(-1000,4250)
plt.show()

In [ ]:
sns.distplot(Difference[2050],vertical=True,color='w')
plt.xlabel('Relative frequency')
plt.ylabel('2050 - Cumulative difference (kton)')
plt.ylim(-1000,4250)

### Let's now evaluate the scatter plot trends over time

In [ ]:
for col in yearlySilver.iloc[:, list(range(1)) + [-1]]:
    plt.scatter(silverCellDistribution2, yearlySilver[col], s = 2, label = str(col))
    plt.plot(np.unique(silverCellDistribution2), 
    np.poly1d(np.polyfit(silverCellDistribution2[0], yearlySilver[col], 1))(np.unique(silverCellDistribution2)), color='k',
            linewidth=1, label = 'Linear regression')
    plt.xlabel('$PV_{Ag}R_{d}$')
    plt.ylabel('Yearly PV silver demand (kton)')
    plt.legend()
    plt.show()

In [ ]:
for col in yearlySilver.iloc[:, list(range(1)) + [-1]]:
    plt.scatter(PVPC2050IncreasePaceDist, yearlySilver[col], c = 'g',s = 2, label = str(col))
    plt.plot(np.unique(PVPC2050IncreasePaceDist), 
    np.poly1d(np.polyfit(PVPC2050IncreasePaceDist[0], yearlySilver[col], 1))(np.unique(PVPC2050IncreasePaceDist)), color='k',
            linewidth=1, label = 'Linear regression')
    plt.xlabel('$PC_{PV}R_{i}$')
    plt.ylabel('Yearly PV silver demand (kton)')
    plt.legend()
    plt.show()

In [ ]:
for col in yearlySilver.iloc[:, list(range(1)) + [-1]]:
    plt.scatter(otherAgDist.loc[col], yearlySilver[col], c = 'b',s = 2, label = str(col))
    plt.plot(np.unique(otherAgDist.loc[col]), 
    np.poly1d(np.polyfit(otherAgDist.loc[col].values, yearlySilver[col], 1))(np.unique(otherAgDist.loc[col].values)), color='k',
            linewidth=1, label = 'Linear regression')
    plt.xlabel('$nPV_{Ag}$ (kton)')
    plt.ylabel('Yearly PV silver demand (kton)')
    plt.legend()
    plt.show()

### Sensitivity indices plots

In [ ]:
matplotlib.rcParams['figure.figsize']  = (7.0,5.0)
df_l = [S_silverCell_df,S_PVPC_df,T_PVPC_df-S_PVPC_df,S_otherAg_df]
df_names = ['$S_{PV_{Ag}R_{d}}$','$S_{PC_{PV}R_{i}}$','$S_{PV_{Ag}R_{d},PC_{PV}R_{i}}$','$S_{nPV_{Ag}}$']
markers = ['o','v','s','P']
for id, dfl in enumerate(df_l):
    for c in dfl:
        plt.scatter(c,dfl[c].mean(),color='k',marker=markers[id],label = df_names[id] if c == 2018 else "") 
plt.legend(prop={'size': 15})
plt.xlabel('Year')
plt.show()

In [ ]:
for dl in df_l: #Print out the confidence interval for the sensitivity indices in the year 2050
    print(stats.t.interval(0.95, len(dl[2050])-1, loc=np.mean(dl[2050]), scale=stats.sem(dl[2050])))

### 3D plots to the year 2050

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
surf=ax.plot_trisurf(df20503d['silverCell'], df20503d['PVPC'], df20503d['yearlySilver'], cmap=plt.cm.jet, linewidth=0.01)
ax.set_xlabel('$PV_{Ag}R_{d}$')
ax.set_ylabel('$PC_{PV}R_{i}$')
ax.set_zlabel('Yearly silver demand 2050 (kton)')
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
surf=ax.plot_trisurf(df20503d['silverCell'], df20503d['PVPC'], df20503d['CumulativeSilver'], cmap=plt.cm.jet, linewidth=0.01)
ax.set_xlabel('$PV_{Ag}R_{d}$')
ax.set_ylabel('$PC_{PV}R_{i}$')
ax.set_zlabel('Cumulative Silver demand 2050 (kton)')
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (5,5)
plt.scatter(df20503d['PVPC'][df20503d['yearly Delta']>0],df20503d['silverCell'][df20503d['yearly Delta']>0],c='g',s=2,
           label='Viable_2050')
plt.scatter(df20503d['PVPC'][df20503d['yearly Delta']<0],df20503d['silverCell'][df20503d['yearly Delta']<0],c='r',s=2,
           label='Non-viable_2050')
plt.xlabel('$PC_{PV}R_{i}$')
plt.ylabel('$PV_{Ag}R_{d}$')
plt.legend()
plt.show()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (5,5)
plt.scatter(df20503d['PVPC'][df20503d['Difference']>0],df20503d['silverCell'][df20503d['Difference']>0],c='g',s=2,
           label='Feasible_2050')
plt.scatter(df20503d['PVPC'][df20503d['Difference']<0],df20503d['silverCell'][df20503d['Difference']<0],c='r',s=2,
           label='Non-feasible_2050')
plt.xlabel('$PC_{PV}R_{i}$')
plt.ylabel('$PV_{Ag}R_{d}$')
plt.legend()
plt.show()